# Laboratorium 4 - rekomendacje dla portali informacyjnych

## Przygotowanie

 * pobierz i wypakuj dataset: https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
   * więcej możesz poczytać tutaj: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-microsoft-news
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab4`
 * zainstaluj potrzebne biblioteki:
 `pip install nltk sklearn`

## Część 1. - przygotowanie danych

In [36]:
# importujemy wszystkie potrzebne pakiety

import codecs
from collections import defaultdict # mozesz uzyc zamiast zwyklego slownika, rozwaz wplyw na czas obliczen
import math
import re
from string import punctuation

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# mozesz uzyc do obliczania najbardziej podobnych tekstow zamiast liczenia "na piechote"
# ale pamietaj o dostosowaniu formatu danych
from sklearn.neighbors import NearestNeighbors

# na potrzeby wizualizacji
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /Users/barto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
# definiujemy potrzebne zmienne

PATH = './MINDsmall_train'
STOPWORDS = set(stopwords.words('english'))

In [38]:
# wczytujemy metadane artykułów

def parse_news_entry(entry):
    news_id, category, subcategory, title, abstract = entry.split('\t')[:5]
    return {
        'news_id': news_id,
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'abstract': abstract
    }

def get_news_metadata():
    with codecs.open(f'{PATH}/news.tsv', 'r', 'UTF-8') as f:
        raw = [x for x in f.read().split('\n') if x]
        parsed_entries = [parse_news_entry(entry) for entry in raw]
        return {x['news_id']: x for x in parsed_entries}

news = get_news_metadata()
news_ids = sorted(list(news.keys()))
news_indices = {x[1]: x[0] for x in enumerate(news_ids)}
print(len(news))

51282


In [39]:
def parse_history_entry(entry):
    _id, user_id, _time, history, _impressions = entry.split('\t')
    history = [x for x in history.split() if x]
    return user_id, history

def get_users_history():
    with codecs.open(f'{PATH}/behaviors.tsv', 'r', 'UTF-8') as f:
        lines = [x for x in f.read().split('\n') if x]
        entries = [parse_history_entry(x) for x in lines]
        return dict(entries)

users_history = get_users_history()
test_users = 'U53231', 'U89744', 'U10045', 'U92486', 'U70879'
print(len(users_history))

50000


## Część 2. - TF-IDF

In [ ]:
from typing import Literal

# normalizujemy teksty na potrzeby dalszego przetwarzania


def preprocess_text(text: str) -> list[str]:
    # zamieniamy wszystkie ciagi bialych znakow na pojedyncze spacje
    text = " ".join(text.split())
    # usuwamy znaki interpunkcyjne
    for p in punctuation:
        text = text.replace(p, "")
    # usuwamy wszystkie liczby
    for n in "0123456789":
        text = text.replace(n, "")
    # podmieniamy wszystkie wielkie litery
    text = text.lower()
    # dzielimy na tokeny
    tokens = set(text.split())
    # usuwamy stopwords
    tokens -= STOPWORDS

    return list(tokens)


def stem_texts(
    corpus: list[str],
    method: Literal["Lancaster", "Porter", "Snowball"] = "Lancaster",
) -> list[list[str]]:
    # stemmer = ...  # przetestuj rozne stemmery
    match method:
        case "Lancaster":
            stemmer = LancasterStemmer()
        case "Porter":
            stemmer = PorterStemmer()
        case "Snowball":
            stemmer = SnowballStemmer(language="english")
    return [[stemmer.stem(word) for word in preprocess_text(text)] for text in corpus]


texts = [news[news_id]["abstract"] for news_id in news_ids]
stemmed_texts = stem_texts(texts, method="Porter")
print(len(stemmed_texts))

51282


In [28]:
# porownajmy teksty przed i po przetworzeniu

print(texts[2] + '\n')
print(' '.join(stemmed_texts[2]))

"I think we have a really good team, and a team that can really do some special, good things because that group is very close in there." - Brian Schmetzer

special thing group team brian realli close think schmetzer good


In [ ]:
# tworzymy liste wszystkich slow w korpusie

def get_all_words_sorted(corpus: list[list[str]]) -> list[str]:
    # generujemy posortowana alfabetycznie liste wszystkich slow (tokenow)
    all_tokens = set()
    for text in corpus:
        all_tokens |= set(text)
    return sorted(all_tokens)

wordlist = get_all_words_sorted(stemmed_texts)
word_indices = {x[1]: x[0] for x in enumerate(wordlist)}
print(len(wordlist))

41884


In [ ]:
# obliczamy liczbe tekstow, w ktorych wystapilo kazde ze slow
# pamietaj, ze jesli slowo wystapilo w danym tekscie wielokrotnie, to liczymy je tylko raz


def get_document_frequencies(
    corpus: list[list[str]],
    wordlist: list[str],
) -> defaultdict[str, int]:
    # return {word -> count}
    f = defaultdict(lambda: 0)
    for text in corpus:
        for word in set(text):
            f[word] += 1
    return f


document_frequency = get_document_frequencies(stemmed_texts, wordlist)

In [ ]:
# obliczamy liczbe wystapien kazdego slowa w kazdym tekscie


def get_term_frequencies(
    corpus: list[list[str]],
    news_indices: list[str],
) -> defaultdict[str, defaultdict[str, int]]:
    # return {news_id -> {word -> count}}
    f = defaultdict(lambda: defaultdict(lambda: 0))
    for news_id, text in zip(news_indices, corpus):
        for word in text:
            f[news_id][word] += 1
    return f


term_frequency = get_term_frequencies(stemmed_texts, news_indices)

In [46]:
# sprawdzmy wyniki

term_frequency[news_ids[2]]

defaultdict(<function __main__.get_term_frequencies.<locals>.<lambda>.<locals>.<lambda>()>,
            {'special': 1,
             'thing': 1,
             'group': 1,
             'team': 1,
             'brian': 1,
             'realli': 1,
             'close': 1,
             'think': 1,
             'schmetzer': 1,
             'good': 1})

In [ ]:
# obliczamy metryke tf_idf
from math import log


def calculate_tf_idf(
    term_frequency: defaultdict[str, defaultdict[str, int]],
    document_frequency: defaultdict[str, int],
    corpus_size: int,
) -> defaultdict[str, defaultdict[str, float]]:
    # return {news_id -> {word -> tf_idf}}
    tf_idf = defaultdict(lambda: defaultdict(lambda: 0.0))
    for news_id in term_frequency:
        for word in term_frequency[news_id]:
            tf_idf[news_id][word] = term_frequency[news_id][word] * log(corpus_size / document_frequency[word])
    return tf_idf


tf_idf = calculate_tf_idf(term_frequency, document_frequency, len(news_ids))

In [51]:
# sprawdzmy wyniki

tf_idf[news_ids[2]]

defaultdict(<function __main__.calculate_tf_idf.<locals>.<lambda>.<locals>.<lambda>()>,
            {'special': 4.765161897298483,
             'thing': 4.111693200556713,
             'group': 4.261685870235309,
             'team': 3.078678194374417,
             'brian': 5.624739267315748,
             'realli': 4.853630545286091,
             'close': 3.9244235881453897,
             'think': 4.360459856758821,
             'schmetzer': 9.458800731274183,
             'good': 4.0427003290697625})

## Część 3. - Podobieństwo tekstów

In [52]:
# obliczmy odleglosc miedzy dwoma artykulami
# przetestuj rozne metryki odleglosci i wybierz najlepsza
from math import sqrt

def calculate_distance(
    tf_idf: defaultdict[str, defaultdict[str, float]],
    id1: str,
    id2: str,
    method: Literal["euclidean"] = "euclidean",
) -> float:
    match method:
        case "euclidean":
            d = sqrt(sum(
                (tf_idf[id1][word] - tf_idf[id2][word]) ** 2
                for word in set(tf_idf[id1].keys()) | set(tf_idf[id2].keys())
            ))
        case _:
            raise ValueError(f"Unknown method: {method}")

    return d


calculate_distance(tf_idf, news_ids[2], news_ids[1])

40.992934972419235

In [ ]:
# funkcja pomocnicza do wyswietlania artykulow
def print_news_entry(n_id: str, corpus: dict[str, dict[str, str]]):
    print(f'id: {n_id}\n\ttitle: {corpus[n_id]["title"]}\n\ttext: {corpus[n_id]["abstract"]}')

print_news_entry('N42782', news)

id: N42782
	title: Three takeaways from Yankees' ALCS Game 5 victory over the Astros
	text: The Yankees kept hope alive thanks to some impressive starting pitching and a pair of early home runs.


In [ ]:
# wyznaczmy k najpodobniejszych tekstow do danego
# pamietaj o odpowiedniej kolejnosci sortowania w zaleznosci od wykorzystanej metryki
# pamietaj, zeby wsrod podobnych tekstow nie bylo danego


def get_k_most_similar_news(
    tf_idf: defaultdict[str, defaultdict[str, float]],
    n_id: str,
    k: int,
) -> list[str]:
    ...


def print_k_most_similar_news(
    tf_idf: defaultdict[str, defaultdict[str, float]],
    n_id: str,
    k: int,
    corpus: dict[str, dict[str, str]],
):
    similar = get_k_most_similar_news(tf_idf, n_id, k)
    print_news_entry(n_id, corpus)
    print(f"\n{k} most similar:")
    for s_id in similar:
        print_news_entry(s_id, corpus)


print_k_most_similar_news(tf_idf, news_ids[42337], 5, news)

## Część 4. - Profile użytkowników

In [ ]:
# oblicz srednia z wektorow tf-idf artykulow o zadanych id-kach
def calculate_average_vector(tf_idf, news_ids: list[str]) -> dict[str, float]:
    raise NotImplementedError()

# wykorzystaj powyzsza funkcje, by policzyc wektor kazdego uzytkownika
def calculate_users_vectors(tf_idf, users_history) -> dict[str, list]:
    raise NotImplementedError()

user_vectors = calculate_users_vectors(tf_idf, users_history)


In [ ]:
# sprawdz wyliczony profil dla przykladowego uzytkownika
print(sorted([(k,v) for k,v in user_vectors[test_users[0]].items() if v], key=lambda x: -x[1]))

In [ ]:
# skorzystajmy ze znanej juz biblioteki, by to lepiej zwizualizowac
def plot_vector(tf_idf_vector):
    wordcloud = WordCloud(random_state=42, background_color='black', colormap='Set2')
    wordcloud.generate_from_frequencies(frequencies=tf_idf_vector)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

plot_vector(user_vectors[test_users[0]])

## Część 5. - Rekomendacje dla użytkowników

In [ ]:
# wykorzystujac wektory tresci i profile uzytkownikow,
#   wygeneruj liste k artykulow najlepiej dopasowanych do uzytkownika
#   pamietaj o odsianiu artykulow, ktore uzytkownik juz kliknal

def recommend(tf_idf, user_id, news, users_history, k):
    raise NotImplementedError()

# dla wybranego uzytkownika, korzystajac z juz zaimplementowanych funkcji,
#   pokaz jego historie, profil (wordcloud) i rekomendacje

## Część 6. - Ocena jakości

In [ ]:
# jaccard index to metryka podobienstwa zbiorow, lekko ja zmodyfikujemy
# przeciecie wektorow to minimum po kazdej wspolrzednej
# unia wektorow to maksimum po kazdej wspolrzednej
# jaccard index to iloraz sum tych dwoch wartosci

def jaccard(v1, v2):
    raise NotImplementedError()


In [ ]:
# dla kazdego uzytkownika wygeneruj k-elementowa rekomendacje
# policz jaccard index miedzy wektorem uzytkownika a srednim wektorem elementow z rekomendacji
# porownaj wyniki dla dwoch roznych k i dwoch roznych metryk podobienstwa

raise NotImplementedError()